In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.7 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.4 MB/s eta 0:00:00 MB/s eta 0:00:01


In [2]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 22.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:01:01


In [1]:
import pandas as pd
import re
import gensim # thư viện NLP
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from pandas import DataFrame
import re

In [2]:
df_train_feature = pd.read_csv("../../data//raw/vn_hate_speech/02_train_text.csv",encoding="UTF8")

In [3]:
df_train_feature.head()

,id,free_text
0,train_blzwlqjqra,vừa rẻ vừa hịn. tội gì k mua. Cầm Kiều Oanh <U...
1,train_pcndwphict,Tại sao nhìn con dao bạn không nghĩ đến việc s...
2,train_abhvpptixv,tao lấy trứng dái đập hả m con lol :)))
3,train_tervdkytrt,Trang Bằng shared a post to the group: MUA BÁN...
4,train_qceexodeie,Set bộ LV .\nSz : s-m-l .\n📌340 Thảo Mi Lê add...


In [4]:
df_train_label = pd.read_csv("../../data/raw/vn_hate_speech/03_train_label.csv",encoding="UTF8")

In [5]:
df_train_label.head()

,id,label_id
0,train_xeokpxcudd,0
1,train_zeahsydrif,0
2,train_bpschfxvea,0
3,train_jadogeuwsz,0
4,train_ijfaidtjej,0


In [6]:
stop_words = pd.read_csv("../../data/vi_stopwords.txt")

In [7]:
with open('../../data/vi_stopwords.txt') as f:
    stop_words = f.readlines()
    f.close()

In [8]:
for idx,word in enumerate(stop_words):
    word = re.sub("\n","",word)
    word = re.sub("[ ]","_",word)
    stop_words[idx] = word

In [9]:
with open('../../data/vi_stopwords_segmented.txt','w') as f:
    temp = ",".join(stop_words)
    f.write(temp)
    f.close()

In [10]:
from collections import Counter

In [11]:
def remove_html_tag(batch):
    batch = re.sub("(<.*>)","", batch)
    batch = re.sub("(url photos from .*? post)", "", batch)
    batch = re.sub("updated .*? status", "", batch)
    return batch

In [12]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\\\n\(\)]'

def remove_special_characters(batch):
    batch = re.sub(chars_to_ignore_regex, '', batch)
    return batch

In [13]:
def remove_number_characters(batch):
    batch = re.sub("\d+",'', batch)
    return batch

In [14]:
def remove_any_special_left(batch):
    # Loại bỏ các kí tự đặc biệt
    batch = gensim.utils.simple_preprocess(batch)
    batch = ' '.join(batch)
    # Sau khi loại bỏ kí tự đặc biệt thì file sẽ bị mất nhận diện chuỗi -> thêm dấu nháy vào chuỗi
    return batch

In [15]:
df_train_feature['free_text'] = df_train_feature['free_text'].apply(lambda x : x.upper())
df_train_feature['free_text'] = df_train_feature['free_text'].apply(remove_html_tag)
df_train_feature['free_text'] = df_train_feature['free_text'].apply(remove_special_characters)
df_train_feature['free_text'] = df_train_feature['free_text'].apply(remove_number_characters)
df_train_feature['free_text'] = df_train_feature['free_text'].apply(remove_any_special_left)

In [16]:
df_train_feature.head(10)

,id,free_text
0,train_blzwlqjqra,vừa rẻ vừa hịn tội gì mua cầm kiều oanh trang ...
1,train_pcndwphict,tại sao nhìn con dao bạn không nghĩ đến việc s...
2,train_abhvpptixv,tao lấy trứng dái đập hả con lol
3,train_tervdkytrt,trang bằng shared post to the group mua bán on...
4,train_qceexodeie,set bộ lv sz sml thảo mi lê added new photos p...
5,train_hmmegjkmpi,chị gio màu váy và đọc mã màu chọn màu
6,train_appnmrhqqi,ngọc trần cherry fashion shop sale túi sập sàn...
7,train_oflzkbmdbm,mụn thịt mụn cám quanh mắtai bị mụn thịt mắt q...
8,train_wqpclypprz,dũng nguyễn thành long nguyễn vũ hạnh nhi
9,train_tqruvghekx,nhà có trẻ con thì dầu_tràm là nên có trong nh...


In [17]:
from vncorenlp import VnCoreNLP

In [18]:
rdrsegmenter = VnCoreNLP("../../vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [19]:
def segment_text(batch):
    batch = rdrsegmenter.tokenize(batch)
    if len(batch) == 0:
        return ""
    else:
        return " ".join(batch[0]).strip()

In [20]:
df_train_feature.head()

,id,free_text
0,train_blzwlqjqra,vừa rẻ vừa hịn tội gì mua cầm kiều oanh trang ...
1,train_pcndwphict,tại sao nhìn con dao bạn không nghĩ đến việc s...
2,train_abhvpptixv,tao lấy trứng dái đập hả con lol
3,train_tervdkytrt,trang bằng shared post to the group mua bán on...
4,train_qceexodeie,set bộ lv sz sml thảo mi lê added new photos p...


In [21]:
df_train_feature['free_text'] = df_train_feature['free_text'].apply(segment_text)

In [22]:
df_train_feature['free_text'] = df_train_feature['free_text'].apply(lambda x : re.sub("xxx","",x).strip())

In [23]:
count1 = Counter(" ".join(df_train_feature['free_text']).split()).most_common(20)

In [24]:
dfc = pd.DataFrame.from_dict(count1)
dfc = dfc.rename(columns={0: "words", 1 : "count"})
dfc

,words,count
0,vl,4942
1,photos,3961
2,có,3385
3,post,3150
4,cho,2874
5,from,2763
6,là,2653
7,và,2186
8,không,1918
9,đi,1858


In [25]:
train_final = pd.merge(df_train_feature,df_train_label,on=['id'])

In [26]:
train_final

,id,free_text,label_id
0,train_blzwlqjqra,vừa rẻ vừa hịn tội gì mua cầm_kiều oanh trang ...,0
1,train_pcndwphict,tại_sao nhìn con dao bạn không nghĩ đến việc s...,0
2,train_abhvpptixv,tao lấy trứng dái đập hả con lol,2
3,train_tervdkytrt,trang bằng shared post to the group mua_bán on...,0
4,train_qceexodeie,set bộ lv sz sml thảo mi lê_added new photos p...,0
...,...,...,...
20340,train_owkyoabvtd,hoàng lý chịu_khó coi cm đi kk,0
20341,train_kucaytiqfn,hương quỳnh tôm bi,0
20342,train_pcmlcrbdso,minh tầm dth không_anh,0
20343,train_tmgipnrqqg,phuong phan mới bầu lứa thui cũng có cho đẻ nh...,0


In [28]:
import numpy as np
indices = np.where(train_final['free_text'] == "")

In [30]:
indices

(array([ 2090,  2409,  2460,  2642,  3217,  3248,  5678,  6231, 11044,
        12087, 12662, 17610]),)

In [32]:
train_final.drop(indices[0],axis=0,inplace=True)

In [35]:
train_dataset = train_final.sample(frac=0.8,random_state=42)
val_dataset = train_final.drop(train_dataset.index)

In [36]:
train_dataset.to_csv('../../data/processed/train.csv')
val_dataset.to_csv('../../data/processed/val.csv')

In [311]:
df_test_feature = pd.read_csv("../../data/raw/vn_hate_speech/04_test_text.csv",encoding="UTF8")

In [312]:
df_test_feature.head()

,id,free_text
0,test_ryhoydgtcg,Mình thanh lý giúp khách bức tranh Tranh thêu ...
1,test_yqasbugonw,Buồn Bất Cần is feeling alone.
2,test_etnlgoxldg,Hàng mới về Nga Nguyen is with Tran Cuong and ...
3,test_fhabnngpzi,Mệt Vl :))
4,test_wqkbbeobym,Minh Chí nhán nhí vl


In [313]:
df_test_feature['free_text'] = df_test_feature['free_text'].apply(lambda x : x.upper())
df_test_feature['free_text'] = df_test_feature['free_text'].apply(remove_html_tag)
df_test_feature['free_text'] = df_test_feature['free_text'].apply(remove_special_characters)
df_test_feature['free_text'] = df_test_feature['free_text'].apply(remove_number_characters)
df_test_feature['free_text'] = df_test_feature['free_text'].apply(remove_any_special_left)
df_test_feature['free_text'] = df_test_feature['free_text'].apply(segment_text)
df_test_feature['free_text'] = df_test_feature['free_text'].apply(lambda x : re.sub("xxx","",x).strip())
indices = np.where(df_test_feature['free_text'] == "")
df_test_featuretrain_final.drop(indices[0],axis=0,inplace=True)
df_test_feature.to_csv('../../data/processed/test.csv')